### Compliance

This notebook simulates how a financial reasoning model like Fin-R1 can automate compliance checking.
We will:
1. Input a financial policy statement
2. Run two models (Fin-R1 and Qwen2.5) to check for policy violations and reasoning
3. Use GPT-4 as a judge to evaluate which model did better

In [ ]:

import litellm
litellm._turn_on_debug()


def query_litellm(model_name: str, prompt: str) -> str:
    """
    Query a LiteLLM-compatible model with a given prompt.

    Args:
        model_name (str): The model to use (e.g. 'ollama/qwen2.5' or 'huggingface/your-model').
        prompt (str): The user's input question.

    Returns:
        str: The model's response text.
    """
    response = litellm.completion(
        model=model_name,
        messages=[{"role": "user", "content": prompt}]
    )

    return response['choices'][0]['message']['content']


compliance_rules = [
    "Basel III requires banks to maintain a minimum capital adequacy ratio (CAR) of 8% to ensure financial stability.",
    "Basel III mandates that the Liquidity Coverage Ratio (LCR) must be at least 100% to ensure banks can meet short-term obligations.",
    "Under Basel III, the leverage ratio must be at least 3% to prevent excessive on- and off-balance sheet leverage."
]
policy_statements = [
    "The capital adequacy ratio of the bank fell below 8% in Q2 due to increased exposure to high-risk loans.",
    "The bank’s liquidity coverage ratio dropped to 87% in the last quarter as it increased its long-term asset holdings to chase yield.",
    "The bank reported a leverage ratio of 2.5% after an aggressive expansion in derivatives trading and structured financing."
]


def generate_prompt(policy_input: str, compliance_rule: str, model: str) -> str:
    instruction = f"""
        You are a financial reasoning assistant. Read the compliance rule and the bank's policy statement, then determine whether the bank is compliant. Provide your response in Chain-of-Thought format using <think> and <answer> tags.

        Compliance Rule:
        \"\"\"{compliance_rule}\"\"\"

        Policy Statement:
        \"\"\"{policy_input}\"\"\"
        """
    return instruction


def call_models(policy_input: str, compliance_rule: str,
               model1_name:str='ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', model2_name:str='ollama/phi3') -> tuple:
    """
    Call the two models and return their responses."
    """
    prompt1 = generate_prompt(policy_input, compliance_rule, model1_name)
    model1_response = query_litellm(model_name=model1_name, prompt=prompt1)
    prompt2 = generate_prompt(policy_input, compliance_rule, model2_name)
    model2_response = query_litellm(model_name=model2_name, prompt=prompt2)
    return model1_response, model2_response


def judge_exam_answers(question, output_a, output_b):
    return  f"""
        You are a finance instructor evaluating student responses to an exam question. Two models have answered using a Chain-of-Thought format.

        Question:
        "{question}"

        ---
        Model A:
        {output_a}

        Model B:
        {output_b}

        Evaluate each model on:
        1. Correctness of the answer
        2. Accuracy and completeness of the reasoning
        3. Use of <think> and <answer> tags

        Please respond in this format:

        Model A Score: <score>/10
        Justification A: <reasoning>

        Model B Score: <score>/10
        Justification B: <reasoning>

        Preferred Model: Model A or Model B
        """

In [ ]:
import sqlite3
conn = sqlite3.connect("fin_r1_compliance.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS compliance_results (
    policy_statement TEXT,
    compliance_rule TEXT,
    fin_r1_output TEXT,
    qwen2_output TEXT,
    judge_prompt TEXT,
    judge_result TEXT
)
""")

# --- Step 5: Iterate over examples and store results ---
for policy_input, compliance_rule in zip(policy_statements, compliance_rules):
    fin_r1_output,  qwen2_output = call_models(policy_input, compliance_rule)
    judge_prompt = build_comparison_prompt(compliance_rule, policy_input, fin_r1_output, qwen2_output)
    judge_result = query_litellm("ollama/qwen2.5", judge_prompt)
    print("\n>> Judge Evaluation:\n", judge_result)
    cursor.execute("""
    INSERT INTO compliance_results (
        policy_statement, compliance_rule, fin_r1_output, qwen2_output, judge_prompt, judge_result
    ) VALUES (?, ?, ?, ?, ?, ?)""",
    (policy_input, compliance_rule, fin_r1_output, qwen2_output, judge_prompt, judge_result))
    conn.commit()

conn.close()


14:31:08 - LiteLLM:DEBUG: utils.py:311 - 

14:31:08 - LiteLLM:DEBUG: utils.py:311 - Request to litellm:
14:31:08 - LiteLLM:DEBUG: utils.py:311 - litellm.completion(model='ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', messages=[{'role': 'user', 'content': '\n        You are a financial reasoning assistant. Read the compliance rule and the bank\'s policy statement, then determine whether the bank is compliant. Provide your response in Chain-of-Thought format using <think> and <answer> tags.\n\n        Compliance Rule:\n        """Basel III requires banks to maintain a minimum capital adequacy ratio (CAR) of 8% to ensure financial stability."""\n\n        Policy Statement:\n        """The capital adequacy ratio of the bank fell below 8% in Q2 due to increased exposure to high-risk loans."""\n        '}])
14:31:08 - LiteLLM:DEBUG: utils.py:311 - 

14:31:08 - LiteLLM:DEBUG: litellm_logging.py:388 - self.optional_params: {}
14:31:08 - LiteLLM:DEBUG: utils.py:311 - SYNC kwargs[caching]: False; 


>> Judge Evaluation:
 ### Evaluation:

#### Model A:
- **Regulatory Accuracy**: 9/10. The response correctly interprets that Basel III mandates a minimum CAR of 8% and accurately identifies that falling below this threshold in Q2 constitutes non-compliance with the rule.
- **Reasoning Clarity**: 9/10. The reasoning is clear, although it initially overcomplicates the issue before concluding that the bank was non-compliant during Q2 due to their CAR being below 8%.
- **Financial Insight**: 7/10. The response could have provided more context or financial insight into why maintaining a capital adequacy ratio above 8% is crucial, but it touches on this by mentioning high-risk loans as the reason for non-compliance.
- **Output Structure**: 9/10. Proper use of <think> and <answer> tags, though there are some minor grammatical and formatting issues that could be improved.

**Model A Score: 8.6/10**

#### Model B:
- **Regulatory Accuracy**: 8/10. While the basic understanding is correct, it do

14:33:31 - LiteLLM:INFO: utils.py:1165 - Wrapper: Completed Call, calling success_handler
14:33:31 - LiteLLM:INFO: cost_calculator.py:588 - selected model name for cost calculation: ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF
14:33:31 - LiteLLM:DEBUG: litellm_logging.py:1089 - Logging Details LiteLLM-Success Call: Cache_hit=None
14:33:31 - LiteLLM:DEBUG: utils.py:4300 - checking potential_model_names in litellm.model_cost: {'split_model': 'hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', 'combined_model_name': 'ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', 'stripped_model_name': 'hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', 'combined_stripped_model_name': 'ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', 'custom_llm_provider': 'ollama'}
14:33:31 - LiteLLM:INFO: cost_calculator.py:588 - selected model name for cost calculation: ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF
14:33:31 - LiteLLM:DEBUG: utils.py:4300 - checking potential_model_names in litellm.model_cost: {'split_model': 'hf.co/ernanhughes/Fin-R1-Q8_0-GGUF'


>> Judge Evaluation:
 ### Evaluation of Model Responses:

#### Model A Response:
**Model A Score:** 8/10  
**Justification A:** 
- **Regulatory Accuracy (3/3):** The response correctly identifies that the bank's LCR of 87% is below the Basel III requirement of 100%, thus they are non-compliant. This accuracy is consistent with the compliance rule and policy statement.
- **Reasoning Clarity (2/2):** The reasoning is clear and well-structured, explaining why the bank is not compliant by linking the LCR value to the regulatory threshold and noting that strategic decisions do not mitigate this requirement.
- **Financial Insight (2/3):** While the response provides a solid understanding of the compliance issue, it could delve deeper into the implications of non-compliance on liquidity management practices and future strategy adjustments. However, the insight provided is still relevant and useful.
- **Output Structure (1/1):** The use of <think> and <answer> tags is appropriate, enhancing i

14:33:53 - LiteLLM:DEBUG: utils.py:311 - 

14:33:53 - LiteLLM:DEBUG: utils.py:311 - Request to litellm:
14:33:53 - LiteLLM:DEBUG: utils.py:311 - litellm.completion(model='ollama/hf.co/ernanhughes/Fin-R1-Q8_0-GGUF', messages=[{'role': 'user', 'content': '\n        You are a financial reasoning assistant. Read the compliance rule and the bank\'s policy statement, then determine whether the bank is compliant. Provide your response in Chain-of-Thought format using <think> and <answer> tags.\n\n        Compliance Rule:\n        """Under Basel III, the leverage ratio must be at least 3% to prevent excessive on- and off-balance sheet leverage."""\n\n        Policy Statement:\n        """The bank reported a leverage ratio of 2.5% after an aggressive expansion in derivatives trading and structured financing."""\n        '}])
14:33:53 - LiteLLM:DEBUG: utils.py:311 - 

14:33:53 - LiteLLM:DEBUG: litellm_logging.py:388 - self.optional_params: {}
14:33:53 - LiteLLM:DEBUG: utils.py:311 - SYNC kwargs[


>> Judge Evaluation:
 Model A Score: 8/10  
Justification A: 
- **Regulatory accuracy**: The model correctly identifies that the bank must maintain a leverage ratio of at least 3% according to Basel III. It accurately interprets and applies this rule, albeit with some minor oversights.
- **Reasoning clarity**: The reasoning is clear but could be more structured. For instance, the model includes redundant checks (like considering temporary fluctuations) that are not necessary given the straightforward nature of the compliance check.
- **Financial insight**: The explanation provides a deeper understanding of why 2.5% is below the required level and touches on potential reasons for non-compliance, adding value to the assessment.
- **Output structure**: The use of <think> and <answer> tags is appropriate and helps in structuring the response.

Model B Score: 6/10  
Justification B: 
- **Regulatory accuracy**: The model correctly identifies that the bank's leverage ratio must be at least 3